In [1]:
import pandas as pd
import numpy as np

# Load the decoded dataset
decoded_data = pd.read_excel('properly_decoded_female_farmers_data.xlsx')

# 1. Fill binary variables
binary_vars = ['Neffa', 'Fumées de Tabouna', 'AT en milieu agricole', 'Ménopause']
for var in binary_vars:
    if var in decoded_data.columns:
        decoded_data[var] = decoded_data[var].fillna('non')

# 2. Handle Age ménopause specially
if 'Age ménopause' in decoded_data.columns and 'Ménopause' in decoded_data.columns:
    # For women who haven't reached menopause yet
    no_menopause_mask = decoded_data['Ménopause'] == 'non'
    decoded_data.loc[no_menopause_mask, 'Age ménopause'] = 'Pas encore en ménopause'
    
    # For women with menopause but missing age value
    has_menopause_mask = decoded_data['Ménopause'] == 'oui'
    decoded_data.loc[has_menopause_mask & decoded_data['Age ménopause'].isna(), 'Age ménopause'] = 'Non spécifié'

# 3. Fill multivalue indicators
indicator_replacements = {
    'Produits chimiques utilisés': 'aucun produit chimique',
    'Produits biologiques utilisés': 'aucun produit biologique',
    'Engrais utilisés': 'aucun engrais',
    'Contraintes thermiques': 'aucune contrainte thermique',
    'Moyen de transport': 'aucun transport spécifié'
}

for col, replacement in indicator_replacements.items():
    if col in decoded_data.columns:
        decoded_data[col] = decoded_data[col].fillna(replacement)

# 4. Fill gynecological history
if 'Antécédents gynéco' in decoded_data.columns:
    decoded_data['Antécédents gynéco'] = decoded_data['Antécédents gynéco'].fillna('aucun antécédent')

# 5. Fill husband's profession based on marital status
if 'Profession du mari' in decoded_data.columns and 'Situation maritale' in decoded_data.columns:
    # For unmarried women
    single_mask = decoded_data['Situation maritale'].isin(['célibataire', 'divorcée', 'veuve'])
    decoded_data.loc[single_mask & decoded_data['Profession du mari'].isna(), 'Profession du mari'] = 'pas applicable'
    
    # For married women with missing data
    married_mask = decoded_data['Situation maritale'] == 'mariée'
    decoded_data.loc[married_mask & decoded_data['Profession du mari'].isna(), 'Profession du mari'] = 'non spécifié'

# 6. Fill accident mechanism based on accident status
if 'Mécanisme AT' in decoded_data.columns and 'AT en milieu agricole' in decoded_data.columns:
    no_accident_mask = decoded_data['AT en milieu agricole'] == 'non'
    decoded_data.loc[no_accident_mask & decoded_data['Mécanisme AT'].isna(), 'Mécanisme AT'] = "pas d'accident"
    
    # For women with accidents but no specified mechanism
    has_accident_mask = decoded_data['AT en milieu agricole'] == 'oui'
    decoded_data.loc[has_accident_mask & decoded_data['Mécanisme AT'].isna(), 'Mécanisme AT'] = 'non spécifié'

# 7. Handle any remaining NaN values in categorical columns
for col in decoded_data.columns:
    # Skip numerical columns
    if pd.api.types.is_numeric_dtype(decoded_data[col]):
        continue
        
    # Fill remaining NA values based on column context
    if col.startswith('Chemical_') or col.startswith('Bio_') or col.startswith('Fertilizer_') or \
       col.startswith('Thermal_') or col.startswith('Transport_'):
        # Binary indicators should be 0 if missing
        decoded_data[col] = decoded_data[col].fillna(0)
    elif col not in ['Age', 'Nb enfants', 'Nb pers à charge', 'H travail / jour', 
                    'J travail / Sem', 'Poids', 'Taille', 'TAS', 'TAD', 'GAD']:
        # For other categorical columns, fill with 'non spécifié'
        decoded_data[col] = decoded_data[col].fillna('non spécifié')

# Save the completed dataset
decoded_data.to_excel('completed_female_farmers_data.xlsx', index=False)

print("Dataset has been completed with appropriate values for empty fields")
print(f"Dataset has {decoded_data.shape[0]} rows and {decoded_data.shape[1]} columns")

# Display a sample of the data to verify replacements
print("\nSample of completed data:")
print(decoded_data.head())

# Check remaining missing values
missing_counts = decoded_data.isna().sum()
if missing_counts.sum() > 0:
    print("\nRemaining missing values:")
    print(missing_counts[missing_counts > 0])
else:
    print("\nNo missing values remain in the dataset.")

Dataset has been completed with appropriate values for empty fields
Dataset has 80 rows and 37 columns

Sample of completed data:
   N°  Age Situation maritale  Nb enfants  Nb pers à charge  Domicile  \
0   1   60        célibataire           0                 1  monastir   
1   2   50             mariée           1                 1  monastir   
2   3   60             mariée           4                 2  monastir   
3   4   48             mariée           5                 0  monastir   
4   5   43             mariée           3                 2  monastir   

  Niveau socio-économique Tabagisme Neffa Fumées de Tabouna  ... Taille  TAS  \
0                   moyen       non   non               oui  ...    163  120   
1                     bon       non   non               oui  ...    167  120   
2                     bas       non   non               oui  ...    160  150   
3                   moyen       non   non               oui  ...    162  120   
4                     bon      

C:\Users\SelmaB\AppData\Local\Temp\ipykernel_28064\300754696.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Pas encore en ménopause' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  decoded_data.loc[no_menopause_mask, 'Age ménopause'] = 'Pas encore en ménopause'


In [4]:
import pandas as pd
import numpy as np

# Load the completed dataset
print("Loading the dataset...")
data = pd.read_excel('completed_female_farmers_data.xlsx')

print(f"Original data shape: {data.shape}")

# Check Age ménopause column before changes
if 'Age ménopause' in data.columns:
    print("\nBEFORE CHANGES:")
    print("Age ménopause unique values:", data['Age ménopause'].unique())
    print("Age ménopause data type:", data['Age ménopause'].dtype)

# Handle Age ménopause - convert non-numeric values to NaN with explicit verification
if 'Age ménopause' in data.columns:
    print("\nConverting Age ménopause to numeric...")
    
    # Convert to numeric, forcing non-numeric values to NaN
    data['Age ménopause'] = pd.to_numeric(data['Age ménopause'], errors='coerce')
    
    # Verify the conversion
    print("AFTER CHANGES:")
    print("Age ménopause unique values:", data['Age ménopause'].unique())
    print("Age ménopause data type:", data['Age ménopause'].dtype)

# Round numeric variables with explicit verification
integer_variables = ['Age', 'Nb enfants', 'Nb pers à charge', 'Ancienneté agricole', 'J travail / Sem']

for var in integer_variables:
    if var in data.columns:
        print(f"\nProcessing {var}...")
        # Check values before rounding
        print(f"BEFORE: First 5 values of {var}:", data[var].head().values)
        
        # Convert to numeric first to ensure we're working with numbers
        numeric_values = pd.to_numeric(data[var], errors='coerce')
        
        # Round only the numeric values
        mask = ~pd.isna(numeric_values)
        data.loc[mask, var] = numeric_values[mask].round()
        
        # Check values after rounding
        print(f"AFTER: First 5 values of {var}:", data[var].head().values)

# Save the improved dataset with a new name to avoid any write issues
output_file = 'fixed_female_farmers_data.xlsx'
print(f"\nSaving to {output_file}...")
data.to_excel(output_file, index=False)

print(f"Dataset has been successfully saved to {output_file}")
print("The following changes were made:")
print(f"1. Converted non-numeric values in 'Age ménopause' to NaN")
print(f"2. Rounded these integer variables: {', '.join(integer_variables)}")

Loading the dataset...
Original data shape: (80, 37)

BEFORE CHANGES:
Age ménopause unique values: [50 'Pas encore en ménopause' 45 38 47 53 49 55 57 51 48 58 40 64 43 42 30]
Age ménopause data type: object

Converting Age ménopause to numeric...
AFTER CHANGES:
Age ménopause unique values: [50. nan 45. 38. 47. 53. 49. 55. 57. 51. 48. 58. 40. 64. 43. 42. 30.]
Age ménopause data type: float64

Processing Age...
BEFORE: First 5 values of Age: [60 50 60 48 43]
AFTER: First 5 values of Age: [60 50 60 48 43]

Processing Nb enfants...
BEFORE: First 5 values of Nb enfants: [0 1 4 5 3]
AFTER: First 5 values of Nb enfants: [0 1 4 5 3]

Processing Nb pers à charge...
BEFORE: First 5 values of Nb pers à charge: [1 1 2 0 2]
AFTER: First 5 values of Nb pers à charge: [1 1 2 0 2]

Processing Ancienneté agricole...
BEFORE: First 5 values of Ancienneté agricole: [30. 30. 32. 27. 10.]
AFTER: First 5 values of Ancienneté agricole: [30. 30. 32. 27. 10.]

Processing J travail / Sem...
BEFORE: First 5 value